# Fire Detection Project

- Eduardo Guimarães Lino de Paula -- eduardo.guimaraes@ensta-paris.fr
- Filipe Lacerda Benevides -- filipe.lacerda@ensta-paris.fr
- Joaquim Minarelli Gaspar -- joaquim.gaspar@ensta-paris.fr
- Sami Kouzeiha -- sami.kouzeiha@telecom-paris.fr

In this Notebook, we're going to asses the accuracies of the different approaches used during the activity.

We start with the conventional approach of training CNNs in a small partition of the validation set, to check the complexity of our data. 

In [1]:
import torch
import os #for loading the data
from torch.utils.data import DataLoader
import kagglehub

In [2]:
path = kagglehub.dataset_download("abdelghaniaaba/wildfire-prediction-dataset")

In [3]:
#Transformations are created as cited in the report.
from utils import create_weak_aug,create_strong_aug,create_valid_transform

weak = create_weak_aug(size = (224,224))
strong = create_strong_aug(size = (224,224))
valid_transform = create_valid_transform(size=(224,224))

In [4]:
train_path = os.path.join(path,'train')
valid_path = os.path.join(path,'valid')
test_path = os.path.join(path,'test')

In [5]:
#We also use our dataset classes
from datasets import labelled_TensorDataset, unlabelled_TensorDataset


labelled_set = labelled_TensorDataset(name = valid_path, transform=weak)
#for inference
test_set = labelled_TensorDataset(name = test_path, transform=valid_transform)


Premature end of JPEG file


In [ ]:
num_workers = 2

In [9]:


#testing loaders and batches
epochs=20
batch_size = 64
ratio = 4
loss_weight = 1.0
#Optim Parameters
lr = 1e-3
momentum = 0.5
weight_decay = 0.03
nesterov=True



In [10]:
test_loader = DataLoader(test_set,batch_size=batch_size,shuffle=True, pin_memory=True, num_workers=num_workers,persistent_workers=True)


In [11]:
from torch.nn import CrossEntropyLoss

In [12]:
criterion = CrossEntropyLoss()

In [13]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device


device(type='cuda', index=0)

In [ ]:
from utils import epoch_loop_nofix,validate
from itertools import product
from utils import labelset_split
from torchvision.models.resnet import ResNet,BasicBlock
from encoders import Net

models = ['resnet','net']

label_samples = [100,250,500]
for model_str,n_labels in product(models,label_samples):
    save_path = model_str+"_"+str(n_labels)+".pth.tar"
    print("We are training "+save_path)
    labelled_filtered_set,val_set = labelset_split(labelled_set,n_per_label=n_labels)
    label_loader = DataLoader(labelled_filtered_set,batch_size=batch_size,shuffle=True, pin_memory=True, num_workers=num_workers,persistent_workers=True)
    val_loader = DataLoader(val_set,batch_size=batch_size,shuffle=True, pin_memory=True, num_workers=num_workers,persistent_workers=True)

    if model_str == 'resnet':
        model = ResNet(BasicBlock, [2, 2, 2, 2], num_classes = 2)#Configurations for ResNet18
    if model_str == 'net':
        model = Net()
   
    model.to(device)
    

    optimizer = torch.optim.SGD(model.parameters(), lr=lr,momentum=momentum, weight_decay=weight_decay,nesterov=nesterov)
    

    epoch_loop_nofix(model,label_loader,val_loader,optimizer,criterion,device,epochs,verbose=False,save_path=save_path)

    _,total_acc = validate(model,test_loader,criterion,device,True)

    print("Total accuracy for the convential {} is: {:.2f}%".format(model_str,total_acc))



We are training resnet_100.pth.tar
Total accuracy for the convential resnet is: 26.14%
We are training resnet_250.pth.tar
Total accuracy for the convential resnet is: 14.24%
We are training resnet_500.pth.tar
Total accuracy for the convential resnet is: 14.24%
We are training net_100.pth.tar
Total accuracy for the convential net is: 17.58%
We are training net_250.pth.tar
Total accuracy for the convential net is: 14.24%
We are training net_500.pth.tar
Total accuracy for the convential net is: 14.24%


The accuracy jumps a bit, but most results end up lower than 15%, we'll see how to SSL approaches deal with the task at hand.

## SSL

### Fix Match results

Now we use one of the trained fix match models and evaluate its accuracy for the different quantities of labeled examples used during training.

In [15]:
from utils import epoch_loop_nofix,validate
from itertools import product
from utils import labelset_split
from torchvision.models.resnet import ResNet,Bottleneck

models = ['resnet']
thresh_vals = [0.5]
label_samples = [100,250,500]

for model_str,value,n_labels in product(models,thresh_vals,label_samples):
    save_path = model_str+str(value)+"_"+str(n_labels)+".pth.tar"
    print("We are training "+save_path)
    labelled_filtered_set,val_set = labelset_split(labelled_set,n_per_label=n_labels)
    label_loader = DataLoader(labelled_filtered_set,batch_size=batch_size,shuffle=True, pin_memory=True, num_workers=num_workers,persistent_workers=True)
    val_loader = DataLoader(val_set,batch_size=batch_size,shuffle=True, pin_memory=True, num_workers=num_workers,persistent_workers=True)

    model = ResNet(Bottleneck, [3, 4, 6, 3], num_classes = 2)#Configurations for WideResNet50
    checkpoint = torch.load(save_path,weights_only=True)
    model.load_state_dict(checkpoint['model_state_dict'])
   
    model.to(device)
    

    _,total_acc = validate(model,test_loader,criterion,device,True)

    print("Total accuracy for the FixMatch model {} is: {:.2f}%".format(model_str,total_acc))

We are training resnet0.5_100.pth.tar
Total accuracy for the FixMatch model resnet is: 14.24%
We are training resnet0.5_250.pth.tar
Total accuracy for the FixMatch model resnet is: 14.24%
We are training resnet0.5_500.pth.tar
Total accuracy for the FixMatch model resnet is: 14.25%


### End-to-End Learning with Encoder

In this part, we evaluate the encoders made by ResNet18, ResNet34 and ResNet50. The models are fine-tuned in the scarse labeled set.

In [14]:

from encoders import enc_classifier
from utils import epoch_loop_nofix,validate
from itertools import product
from utils import labelset_split

models = ['resnet18','resnet34','resnet50']
label_samples = [100,250,500]

for model_str,n_labels in product(models,label_samples):
    save_path = model_str+"_enc_weak"+".pth.tar"
    if model_str=='resnet18':
        continue
    print("We are fine-tuning {} with {} images p/ label".format(save_path,n_labels))
    labelled_filtered_set,val_set = labelset_split(labelled_set,n_per_label=n_labels)
    label_loader = DataLoader(labelled_filtered_set,batch_size=batch_size,shuffle=True, pin_memory=True, num_workers=num_workers,persistent_workers=True)
    val_loader = DataLoader(val_set,batch_size=batch_size,shuffle=True, pin_memory=True, num_workers=num_workers,persistent_workers=True)

    model = enc_classifier(save_path,resnet=model_str,num_classes=2)
    model.to(device)

    optimizer = torch.optim.SGD(model.parameters(), lr=lr,momentum=momentum, weight_decay=weight_decay,nesterov=nesterov)

    epoch_loop_nofix(model,label_loader,val_loader,optimizer,criterion,device,epochs,verbose=False,save_path='class_'+save_path) 
    

    _,total_acc = validate(model,test_loader,criterion,device,True)

    print("Total accuracy for the encoder {} is: {:.2f}%".format(model_str,total_acc))


We are fine-tuning resnet34_enc_weak.pth.tar with 100 images p/ label
Total accuracy for the encoder resnet34 is: 14.25%
We are fine-tuning resnet34_enc_weak.pth.tar with 250 images p/ label
Total accuracy for the encoder resnet34 is: 14.24%
We are fine-tuning resnet34_enc_weak.pth.tar with 500 images p/ label
Total accuracy for the encoder resnet34 is: 17.57%
We are fine-tuning resnet50_enc_weak.pth.tar with 100 images p/ label
Total accuracy for the encoder resnet50 is: 17.57%
We are fine-tuning resnet50_enc_weak.pth.tar with 250 images p/ label
Total accuracy for the encoder resnet50 is: 17.58%
We are fine-tuning resnet50_enc_weak.pth.tar with 500 images p/ label
Total accuracy for the encoder resnet50 is: 14.24%
